Model için gereken kütüphaneleri yükleriz

In [ ]:
import pandas as pd
import numpy as np
import multiprocessing
from sklearn.metrics import accuracy_score
import nltk
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec,doc2vec
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, plot_confusion_matrix, confusion_matrix
import scikitplot as skplt
from sklearn import metrics,utils
import re

Veri setimizin yüklemesini gerçekleştiririz
Mecvut analizimiz için bir kolonda verinin kendisi, diğer kolonda ise verinin etiketinin bulunması yeterlidir.
text,	labels

	mağazamın ınstagram hesabı için reklam vermeyi...	4
	google şirket reklamı vermek istiyordum bu kon...	2
	ben ınstagram	4
	kolay glsin ben web siteme reklam almak istiyo...	2
	sitemin trafiğini arttırmak için reklam vermey...	0

In [ ]:
train = pd.read_csv('veriseti.csv')
train = train[['text','labels']]

Veri setimizi eğitim ve test veri seti olarak ayırırız

In [ ]:
from sklearn.model_selection import train_test_split
train1,test1 = train_test_split(train, test_size = 0.3, random_state = 42)

Burada veri setimizi modelin analizine daha uygun bir hale getirmek için noktalama işaretlerinden, sembollerden, bozuk karakterlerden ayırıp, büyük harfleri küçük harf haline getiren bir fonksyon oluşturuyoruz ve veri setimize bunu uyguluyoruz.

In [ ]:
import re
def cleanText(input_sentence):
 
  tmp= [word.replace('I','ı') for word in input_sentence.split(' ')]
  tmp= [word.lower() for word in tmp]
  tmp= [word.replace('i̇','i') for word in tmp]
  tmp = [re.sub('[^A-Za-z0-9ğüşıçöiâî]+', ' ', word) for word in tmp]
  tmp = [word.strip(' ') for word in tmp]
  tmp1 =' '.join(tmp)

  return tmp1

In [ ]:
train1['text']=train1['text'].apply(cleanText)
test1['text']=test1['text'].apply(cleanText)

NLTK Kütüphanesi ile veri setimizi analiz öncesi tokenize ediyoruz. Bunu yaparken eğitim ve test veri setlerimizdeki sorgu, sonuna etiketi eklenmiş bir hale dönüşüyor.

In [ ]:
nltk.download('punkt')
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [ ]:
train_tagged = train1.apply(lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.labels]), axis=1)
test_tagged = test1.apply(lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.labels]), axis=1)
tr_tags = [i.tags[0] for i in train_tagged]
te_tags = [i.tags[0] for i in test_tagged] 

Burada lojistik regresyon analizi ve destek vektör makineleri yöntemleri ile sınıflandırma yapan iki ayrı fonksyon tanımlıyoruz.

Lojistik Regresyon Sonuçları Fonksyonu

In [ ]:
def LogReg_Results(train_embeddings,test_embeddings):
  logreg = LogisticRegression(n_jobs=cores, C = 10000, max_iter=10000, tol = 1e-5)
  logreg.fit(train_embeddings, tr_tags)    
  te_tags_predicted = logreg.predict(test_embeddings)
  tr_tags_predicted = logreg.predict(train_embeddings)

  print('Train accuracy %s' % accuracy_score(tr_tags, tr_tags_predicted))
  print('Testing accuracy %s' % accuracy_score(te_tags, te_tags_predicted))
  print('Testing Log-Reg F1 weighted score: {}'.format(f1_score(te_tags, te_tags_predicted, average='weighted')))
  print('Testing Log-Reg F1 macro score: {}'.format(f1_score(te_tags, te_tags_predicted, average='macro')))
  print('Testing Log-Reg F1 macro score class-based:: {}'.format(f1_score(te_tags, te_tags_predicted, average=None)))

C-Destek Vektör Makineleri Sonucu Fonksyonu

In [ ]:
def SVC_Results(train_embeddings,test_embeddings):
  clf = SVC(kernel='linear', C=1).fit(train_embeddings, tr_tags)
  clf_te_tags_predicted = clf.predict(test_embeddings)    
  clf_tr_tags_predicted = clf.predict(train_embeddings)
  
  print('C-SVC Train Score %s\n'%clf.score(train_embeddings, tr_tags))
  print('C-SVC Test Score %s\n'%clf.score(test_embeddings, te_tags))
  print('Testing C-SVC F1 weighted score: {}'.format(f1_score(te_tags, clf_te_tags_predicted, average='weighted')))
  print('Testing C-SVC F1 macro score: {}'.format(f1_score(te_tags, clf_te_tags_predicted, average='macro')))
  print('Testing C-SVC F1 macro score class-based:: {}'.format(f1_score(te_tags, clf_te_tags_predicted, average=None)))


Modelimizi farklı vektör genişliklerinde, farklı iterasyon sayılarında ve farklı model çeşitlerinde birbirine göre eğitmek için iç içe bir döngü oluşturuyoruz. Bu işlem uzun sürmektedir.

In [ ]:
cores = multiprocessing.cpu_count()
for vs in [10,20,30,50,75,100,125,175,200,350,500,750,1000,1250,1500,2000]:  
  for epoch in [1,2,3,4,10]:
    for dm in [0,1]:
      print('\n','setting : ','dm : ', dm, 'vs : ', vs, 'num epochs : ', epoch)
      model_dm = Doc2Vec(dm=dm, vector_size=vs, window=10, negative=5, min_count=1, workers=cores, alpha=0.0061, min_alpha = 0.0001)
      model_dm.build_vocab([x for x in tqdm(train_tagged.values)])

      for ep in range(epoch):
        model_dm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
        model_dm.alpha -= 0.0012
        model_dm.min_alpha = model_dm.alpha
      
      train_embeddings = [] 
      for i in train_tagged:
        embedding = model_dm.infer_vector(i.words, steps=20, alpha = 0.005)
        train_embeddings.append(embedding)        

      test_embeddings = [] 
      for i in test_tagged:
        embedding = model_dm.infer_vector(i.words, steps=20, alpha = 0.005)
        test_embeddings.append(embedding)     

      LogReg_Results(train_embeddings,test_embeddings)
      SVC_Results(train_embeddings,test_embeddings)
      
